In [5]:
import os
from glob import glob
from PIL import Image
from itertools import permutations 
import numpy as np

## Paths

In [6]:
image_path = './'
source_path = image_path + '_indiv_faces/'
in_dirs = glob(source_path + '*' )

img_categories = [i.split('/') [-1] for i in in_dirs]

out_dirs = []
for i in img_categories:
    img_dir = image_path + i
    out_dirs.append(img_dir)
    if not os.path.exists(img_dir):
        os.mkdir(img_dir)    

## Image dimensions and math

In [7]:
n_steps = 6  # hard-coded; number of characters in each row & column of square matrix_img
mask_fraction = .8 # proportion of area around target covered by mask

matrix_img = Image.open(image_path + 'letter_grid_6x6.png')
matrix_size = (min(matrix_img.size), min(matrix_img.size)) # size of speller character matrix image, in pixels
img_format = matrix_img.format

matrix_ctr = (matrix_size[0] // 2, matrix_size[1] // 2)
locs = np.arange(min(matrix_size) / n_steps / 2, min(matrix_size), min(matrix_size) / n_steps).astype(int)
mask_size = round(mask_fraction * locs[1] - locs[0])


## Loop over each image in folder, creating row & column

In [8]:
for idx, i in enumerate(img_categories):
    in_files = glob(in_dirs[idx] + '/*.png')
    out_dir = out_dirs[idx]
    for in_img in in_files:
        tile_img = Image.open(in_img)
        tile_img = tile_img.resize((mask_size, mask_size))
        tile_size = tile_img.size

        row_img = Image.new(tile_img.mode, 
                            (matrix_size[0], tile_size[1])
                            )
        for loc in locs:
            tl_corner = (loc - round(tile_size[0] / 2), 0)
            # print(tl_corner)
            row_img.paste(tile_img, box=tl_corner)
        out_fname = 'face' + os.path.splitext(in_img)[0].split('/')[-1][2:] + '_row.' + img_format
        row_img.save(out_dir + '/' + out_fname)

        col_img = Image.new(tile_img.mode, 
                            (tile_size[0], matrix_size[1])
                            )
        for loc in locs:
            tl_corner = (0, loc - round(tile_size[0] / 2))
            col_img.paste(tile_img, box=tl_corner)
        out_fname = 'face' + os.path.splitext(in_img)[0].split('/')[-1][2:] + '_col.' + img_format
        col_img.save(out_dir + '/' + out_fname)            